# Temperature Diagnostics

Once you copy this repository, feel free to delete this notebook!

## Imports

In [1]:
# Display output of plots directly in Notebook
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import intake
import numpy as np
import pandas as pd
import xarray as xr
from ncar_jobqueue import NCARCluster
from distributed import Client

## Spin up a Cluster

In [2]:
cluster = NCARCluster(memory='10 GB')
cluster.scale(20)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.63:40699,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Data Ingest

In [3]:
# Read in the data using xarray or some other package
data_catalog = intake.open_esm_datastore('data/silver-linings-test.json')

### Subset for 2m Temperature

In [4]:
data_subset = data_catalog.search(frequency='day_1', variable='TREFHT')

In [5]:
data_subset.df

,component,stream,case,member_id,variable,start_time,end_time,time_range,long_name,units,vertical_levels,frequency,path
0,atm,cam.h1,b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.001,1,TREFHT,2035-01-01,2044-12-31,20350101-20441231,Reference height temperature,K,1.0,day_1,/glade/campaign/cesm/development/wawg/WACCM6-T...
1,atm,cam.h1,b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.001,1,TREFHT,2045-01-01,2054-12-31,20450101-20541231,Reference height temperature,K,1.0,day_1,/glade/campaign/cesm/development/wawg/WACCM6-T...
2,atm,cam.h1,b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.001,1,TREFHT,2055-01-01,2064-12-31,20550101-20641231,Reference height temperature,K,1.0,day_1,/glade/campaign/cesm/development/wawg/WACCM6-T...
3,atm,cam.h1,b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.001,1,TREFHT,2065-01-01,2069-12-30,20650101-20691230,Reference height temperature,K,1.0,day_1,/glade/campaign/cesm/development/wawg/WACCM6-T...
4,atm,cam.h1,b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.002,2,TREFHT,2035-01-01,2044-12-31,20350101-20441231,Reference height temperature,K,1.0,day_1,/glade/campaign/cesm/development/wawg/WACCM6-T...
5,atm,cam.h1,b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.002,2,TREFHT,2045-01-01,2054-12-31,20450101-20541231,Reference height temperature,K,1.0,day_1,/glade/campaign/cesm/development/wawg/WACCM6-T...
6,atm,cam.h1,b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.002,2,TREFHT,2055-01-01,2064-12-31,20550101-20641231,Reference height temperature,K,1.0,day_1,/glade/campaign/cesm/development/wawg/WACCM6-T...
7,atm,cam.h1,b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.002,2,TREFHT,2065-01-01,2069-12-31,20650101-20691231,Reference height temperature,K,1.0,day_1,/glade/campaign/cesm/development/wawg/WACCM6-T...
8,atm,cam.h1,b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.003,3,TREFHT,2035-01-01,2044-12-31,20350101-20441231,Reference height temperature,K,1.0,day_1,/glade/campaign/cesm/development/wawg/WACCM6-T...
9,atm,cam.h1,b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.003,3,TREFHT,2045-01-01,2054-12-31,20450101-20541231,Reference height temperature,K,1.0,day_1,/glade/campaign/cesm/development/wawg/WACCM6-T...


### Read in the dictionary of datasets

In [7]:
dsets = data_subset.to_dataset_dict(cdf_kwargs={'chunks':{'time':36}})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.stream.case'


In [8]:
ds = dsets['atm.cam.h1.b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.003']
ds

<xarray.Dataset>
Dimensions:       (lat: 192, lon: 288, lev: 70, ilev: 71, time: 12776, nbnd: 2)
Coordinates:
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lev           (lev) float64 5.96e-06 9.827e-06 1.62e-05 ... 976.3 992.6
  * ilev          (ilev) float64 4.5e-06 7.42e-06 1.223e-05 ... 985.1 1e+03
  * time          (time) object 2035-01-01 00:00:00 ... 2070-01-01 00:00:00
Dimensions without coordinates: nbnd
Data variables: (12/26)
    gw            (lat) float64 dask.array<chunksize=(192,), meta=np.ndarray>
    hyam          (lev) float64 dask.array<chunksize=(70,), meta=np.ndarray>
    hybm          (lev) float64 dask.array<chunksize=(70,), meta=np.ndarray>
    P0            float64 ...
    hyai          (ilev) float64 dask.array<chunksize=(71,), meta=np.ndarray>
    hybi          (ilev) float64 dask.array<chunksize=(71,), meta=np.ndarray>
    ...            ...
    n2ovmr        (time) float64 dask.array<chunksize=(36,), meta=np.ndarray>
    f11vmr        (time) float64 dask.array<chunksize=(36,), meta=np.ndarray>
    f12vmr        (time) float64 dask.array<chunksize=(36,), meta=np.ndarray>
    sol_tsi       (time) float64 dask.array<chunksize=(36,), meta=np.ndarray>
    nsteph        (time) int32 dask.array<chunksize=(36,), meta=np.ndarray>
    TREFHT        (time, lat, lon) float32 dask.array<chunksize=(36, 192, 288), meta=np.ndarray>
Attributes:
    initial_file:            b.e21.BWSSP245cmip6.f09_g17.CMIP6-SSP2-4.5-WACCM...
    source:                  CAM
    model_doi_url:           https://doi.org/10.5065/D67H1H0V
    intake_esm_varname:      ['TREFHT']
    time_period_freq:        day_1
    logname:                 geostrat
    topography_file:         /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv...
    case:                    b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.003
    Conventions:             CF-1.0
    intake_esm_dataset_key:  atm.cam.h1.b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-D...

### Data Operation

First, we set up a few helper functions...

In [9]:
def area_grid(lat, lon):
    """
    Calculate the area of each grid cell
    Area is in square meters
    
    Input
    -----------
    lat: vector of latitude in degrees
    lon: vector of longitude in degrees
    
    Output
    -----------
    area: grid-cell area in square-meters with dimensions, [lat,lon]
    
    Notes
    -----------
    Based on the function in
    https://github.com/chadagreene/CDT/blob/master/cdt/cdtarea.m
    """
    from numpy import meshgrid, deg2rad, gradient, cos
    from xarray import DataArray

    xlon, ylat = meshgrid(lon, lat)
    R = earth_radius(ylat)

    dlat = deg2rad(gradient(ylat, axis=0))
    dlon = deg2rad(gradient(xlon, axis=1))

    dy = dlat * R
    dx = dlon * R * cos(deg2rad(ylat))

    area = dy * dx

    xda = DataArray(
        area,
        dims=["lat", "lon"],
        coords={"lat": lat, "lon": lon},
        attrs={
            "long_name": "area_per_pixel",
            "description": "area per pixel",
            "units": "m^2",
        },
    )
    return xda

In [10]:
def earth_radius(lat):
    '''
    calculate radius of Earth assuming oblate spheroid
    defined by WGS84
    
    Input
    ---------
    lat: vector or latitudes in degrees  
    
    Output
    ----------
    r: vector of radius in meters
    
    Notes
    -----------
    WGS84: https://earth-info.nga.mil/GandG/publications/tr8350.2/tr8350.2-a/Chapter%203.pdf
    '''
    from numpy import deg2rad, sin, cos

    # define oblate spheroid from WGS84
    a = 6378137
    b = 6356752.3142
    e2 = 1 - (b**2/a**2)
    
    # convert from geodecic to geocentric
    # see equation 3-110 in WGS84
    lat = deg2rad(lat)
    lat_gc = np.arctan( (1-e2)*np.tan(lat) )

    # radius equation
    # see equation 3-107 in WGS84
    r = (
        (a * (1 - e2)**0.5) 
         / (1 - (e2 * np.cos(lat_gc)**2))**0.5 
        )

    return r

In [11]:
def center_time(ds):
    """make time the center of the time bounds"""
    ds = ds.copy()
    attrs = ds.time.attrs
    encoding = ds.time.encoding
    tb_name, tb_dim = _get_tb_name_and_tb_dim(ds)
    
    ds['time'] = ds[tb_name].compute().mean(tb_dim).squeeze()
    attrs['note'] = f'time recomputed as {tb_name}.mean({tb_dim})'
    ds.time.attrs = attrs
    ds.time.encoding = encoding
    return ds

def _get_tb_name_and_tb_dim(ds):
    """return the name of the time 'bounds' variable and its second dimension"""
    assert 'bounds' in ds.time.attrs, 'missing "bounds" attr on time'
    tb_name = ds.time.attrs['bounds']        
    assert tb_name in ds, f'missing "{tb_name}"'    
    tb_dim = ds[tb_name].dims[-1]
    return tb_name, tb_dim

In [12]:
def calc_area_weighted_mean(ds):
    ds = center_time(ds.sortby('time'))
    # Do some sort of calculation on the data
    ds_out = (
        (ds.resample(time="AS").mean("time") * da_area).sum(dim=("lat", "lon"))
    ) / total_area
    return ds_out

def convert_to_df(ds):
    return ds.TREFHT.to_series().unstack().T

### Compute the area for the weights

In [ ]:
# area dataArray
da_area = area_grid(ds['lat'], ds['lon'])

# total area
total_area = da_area.sum(['lat','lon'])

### Setup which variables to average

In [ ]:
variables = ['TREFHT']

## Run the computation on each dataset

In [ ]:
xr.set_options(keep_attrs=True)
ds_list = []
for key in dsets.keys():
    ds = dsets[key]
    mean = calc_area_weighted_mean(ds)
    out = mean[variables]
    out.attrs['intake_esm_varname'] = variables
    out.attrs['case'] = ds.case
    ds_list.append(out)

### Here we add additional case information

In [ ]:
cases = []
for ds in ds_list:
    cases.append(ds.case)

In [ ]:
merged_ds = xr.concat(ds_list, dim='case')

In [ ]:
merged_ds['case'] = cases

In [ ]:
merged_ds.persist()

### Use datetime index instead of cftime

In [ ]:
datetimeindex = merged_ds.indexes['time'].to_datetimeindex()

In [ ]:
merged_ds['time'] = datetimeindex

## Save the dataset

In [ ]:
merged_ds.to_netcdf('data/global_average_temperature.nc')